In [6]:
# Import libraries 

import numpy as np              
import pandas as pd         
import seaborn as sns               
import matplotlib.pyplot as plt                    
import transformers 
from transformers import Pipeline
import re
import tqdm
import spacy 

In [16]:
with open("PROJECT_GUTENBERG_EBO.txt", "r", encoding="utf-8") as f:
    content = f.read()
    
print(content)
    

*** START OF THE PROJECT GUTENBERG EBOOK 42078 ***

[Transcriber's Note: letters that were superscripted in the original are
surrounded by brackets and preceded by a caret ^.]



_JANE AUSTEN'S WORKS._


    SENSE AND SENSIBILITY                    2 vols.
    PRIDE AND PREJUDICE                      2 vols.
    MANSFIELD PARK                           2 vols.
    EMMA                                     2 vols.
    NORTHANGER ABBEY                         1 vol.
    PERSUASION                               1 vol.
    LADY SUSAN--THE WATSONS WITH A MEMOIR    1 vol.
    LETTERS                                  1 vol.

[Illustration: J. Austen

_From a Painting in the possession of the Rev. Morland Rice, of
Bramber._]




THE LETTERS

OF

JANE AUSTEN

_Selected from the Compilation of her Great Nephew_

_EDWARD, LORD BRADBOURNE_

BY SARAH CHAUNCEY WOOLSEY


    BOSTON
    LITTLE, BROWN, AND COMPANY
    1908


[Transcriber's Note: While the title page gives credit to Lord
Bradbourne, the 

In [8]:
content[:3000]

'*** START OF THE PROJECT GUTENBERG EBOOK 42078 ***\n\n[Transcriber\'s Note: letters that were superscripted in the original are\nsurrounded by brackets and preceded by a caret ^.]\n\n\n\n_JANE AUSTEN\'S WORKS._\n\n\n    SENSE AND SENSIBILITY                    2 vols.\n    PRIDE AND PREJUDICE                      2 vols.\n    MANSFIELD PARK                           2 vols.\n    EMMA                                     2 vols.\n    NORTHANGER ABBEY                         1 vol.\n    PERSUASION                               1 vol.\n    LADY SUSAN--THE WATSONS WITH A MEMOIR    1 vol.\n    LETTERS                                  1 vol.\n\n[Illustration: J. Austen\n\n_From a Painting in the possession of the Rev. Morland Rice, of\nBramber._]\n\n\n\n\nTHE LETTERS\n\nOF\n\nJANE AUSTEN\n\n_Selected from the Compilation of her Great Nephew_\n\n_EDWARD, LORD BRADBOURNE_\n\nBY SARAH CHAUNCEY WOOLSEY\n\n\n    BOSTON\n    LITTLE, BROWN, AND COMPANY\n    1908\n\n\n[Transcriber\'s Note: While the

In [10]:
pattern = r"(?:^|\n)(CHAPTER\s+[^\n]+|Chapter\s+[^\n]+|LETTER\s+[^\n]+|Book\s+[^\n]+|PART\s+[^\n]+)"
print(pattern)


(?:^|\n)(CHAPTER\s+[^\n]+|Chapter\s+[^\n]+|LETTER\s+[^\n]+|Book\s+[^\n]+|PART\s+[^\n]+)


# TEXT ANALYSIS - 🌻☀️🌞

In [17]:
with open("PROJECT_GUTENBERG_EBO.txt", "r", encoding="utf-8") as f:
    content = f.read()
    
print(content)

*** START OF THE PROJECT GUTENBERG EBOOK 42078 ***

[Transcriber's Note: letters that were superscripted in the original are
surrounded by brackets and preceded by a caret ^.]



_JANE AUSTEN'S WORKS._


    SENSE AND SENSIBILITY                    2 vols.
    PRIDE AND PREJUDICE                      2 vols.
    MANSFIELD PARK                           2 vols.
    EMMA                                     2 vols.
    NORTHANGER ABBEY                         1 vol.
    PERSUASION                               1 vol.
    LADY SUSAN--THE WATSONS WITH A MEMOIR    1 vol.
    LETTERS                                  1 vol.

[Illustration: J. Austen

_From a Painting in the possession of the Rev. Morland Rice, of
Bramber._]




THE LETTERS

OF

JANE AUSTEN

_Selected from the Compilation of her Great Nephew_

_EDWARD, LORD BRADBOURNE_

BY SARAH CHAUNCEY WOOLSEY


    BOSTON
    LITTLE, BROWN, AND COMPANY
    1908


[Transcriber's Note: While the title page gives credit to Lord
Bradbourne, the 

In [20]:
# pattern: newline + ROMAN_NUMERAL + '.' + newline
pattern = r"\n([IVXLCDM]+)\.\s*\n"

parts = re.split(pattern, content)
parts

['*** START OF THE PROJECT GUTENBERG EBOOK 42078 ***\n\n[Transcriber\'s Note: letters that were superscripted in the original are\nsurrounded by brackets and preceded by a caret ^.]\n\n\n\n_JANE AUSTEN\'S WORKS._\n\n\n    SENSE AND SENSIBILITY                    2 vols.\n    PRIDE AND PREJUDICE                      2 vols.\n    MANSFIELD PARK                           2 vols.\n    EMMA                                     2 vols.\n    NORTHANGER ABBEY                         1 vol.\n    PERSUASION                               1 vol.\n    LADY SUSAN--THE WATSONS WITH A MEMOIR    1 vol.\n    LETTERS                                  1 vol.\n\n[Illustration: J. Austen\n\n_From a Painting in the possession of the Rev. Morland Rice, of\nBramber._]\n\n\n\n\nTHE LETTERS\n\nOF\n\nJANE AUSTEN\n\n_Selected from the Compilation of her Great Nephew_\n\n_EDWARD, LORD BRADBOURNE_\n\nBY SARAH CHAUNCEY WOOLSEY\n\n\n    BOSTON\n    LITTLE, BROWN, AND COMPANY\n    1908\n\n\n[Transcriber\'s Note: While th

In [21]:
letters = []

# start from index 1, step by 2: (roman numeral, body)
for i in range(1, len(parts), 2):
    roman_id = parts[i].strip()         # e.g. "I"
    body = parts[i + 1].strip()         # letter text
    
    letters.append({
        "id": roman_id,
        "text": body
    })

print(f"Found {len(letters)} letters.")
print("First letter ID:", letters[0]["id"])
print(letters[0]["text"][:500], "...")


Found 78 letters.
First letter ID: I
STEVENTON, Thursday (January 16, 1796).

I HAVE just received yours and Mary's letter, and I thank you both,
though their contents might have been more agreeable. I do not at all
expect to see you on Tuesday, since matters have fallen out so
unpleasantly; and if you are not able to return till after that day, it
will hardly be possible for us to send for you before Saturday, though
for my own part I care so little about the ball that it would be no
sacrifice to me to give it up for the sake of s ...


In [22]:
letters

[{'id': 'I',
  'text': "STEVENTON, Thursday (January 16, 1796).\n\nI HAVE just received yours and Mary's letter, and I thank you both,\nthough their contents might have been more agreeable. I do not at all\nexpect to see you on Tuesday, since matters have fallen out so\nunpleasantly; and if you are not able to return till after that day, it\nwill hardly be possible for us to send for you before Saturday, though\nfor my own part I care so little about the ball that it would be no\nsacrifice to me to give it up for the sake of seeing you two days\nearlier. We are extremely sorry for poor Eliza's illness. I trust,\nhowever, that she has continued to recover since you wrote, and that you\nwill none of you be the worse for your attendance on her. What a\ngood-for-nothing fellow Charles is to bespeak the stockings! I hope he\nwill be too hot all the rest of his life for it!\n\nI sent you a letter yesterday to Ibthorp, which I suppose you will not\nreceive at Kintbury. It was not very long or

In [23]:
import os

os.makedirs("letters", exist_ok=True)

for idx, letter in enumerate(letters, start=1):
    fname = f"letters/letter_{idx:03d}_{letter['id']}.txt"
    with open(fname, "w", encoding="utf-8") as f:
        f.write(f"{letter['id']}.\n\n{letter['text']}")
